In [1]:
import glob
import pandas as pd

# Creating Sparse Matrix

In [2]:
paths = glob.glob('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/Query results/*.csv')

dfs = [pd.read_csv(filename, index_col=0) for filename in paths]

df = pd.concat(dfs,ignore_index=True)

In [3]:
locations = pd.read_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/locations_updated.csv',index_col=0)
locations['Address'] = locations.index

In [ ]:
timeMatrix = pd.DataFrame(index = set(locations['Address']), columns = set(locations['Address']))
# distanceMatrix = pd.DataFrame(index = set(locations['Address']), columns = set(locations['Address']))
# directionMatrix = pd.DataFrame(index = set(locations['Address']), columns = set(locations['Address']))

i = 0
total = len(df)
for rowNum, row in df.iterrows():
    timeMatrix.loc[row['start'], row['end']] = row['time']
    timeMatrix.loc[row['end'], row['start']] = row['time'] # Not doing this since it makes it easier later to have the Nans
    
#     distanceMatrix.loc[row['start'], row['end']] = row['distance']
#     distanceMatrix.loc[row['end'], row['start']] = row['distance']
    
#     directionMatrix.loc[row['start'], row['end']] = row['directions']
#     directionMatrix.loc[row['end'], row['start']] = row['directions']
    
    i += 1
    if i % 3000 == 0:
        print(round(100*i/total), '% Complete')


i = 0
for i in range(0,len(timeMatrix)):
    timeMatrix.iloc[i,i] = 0
    i += 1
    if i % 10000 == 0:
        print(round(100*i/total), '% Complete')
    


# timeMatrix = pd.read_csv('timeMatrix.csv', index_col=0)
# distanceMatrix = pd.read_csv('distanceMatrix.csv')
# directionMatrix = pd.read_csv('directionMatrix.csv')
# directionMatrix.to_csv('directionMatrix.csv')
    

In [ ]:
timeMatrix.to_csv('sparseTimeMatrix.csv')

In [4]:
sparseMatrix = pd.read_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/sparseTimeMatrix.csv', index_col=0)

In [8]:
sparseMatrix1 = sparseMatrix.iloc[0:len(sparseMatrix)]
sparseMatrix2 = sparseMatrix.iloc[len(sparseMatrix):]

In [9]:
sparseMatrix1.to_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/sparseTimeMatrix1.csv')
sparseMatrix2.to_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/sparseTimeMatrix2.csv')

# Dijkstra's Algorithm

In [ ]:
import multiprocessing as mp
import numpy as np

def dijkstrasThread(graph,locations, rows, processNumber, returnDict):
    print("here")
    results = pd.DataFrame(columns = set(locations))
    print(results)
    for row in rows:
        dist_matrix, predecessors = dijkstra(csgraph=graph, indices=0, directed=False, return_predecessors=True)
        results.loc[locations[row]] = dist_matrix
        

        
#     print(results)

graph = [

[0, 1, 2, 0],

[0, 0, 0, 1],

[0, 0, 0, 3],

[0, 0, 0, 0]

]

graph = csr_matrix(graph)

        
# dijkstrasThread(graph, ['0','1','2','3'],[0,1], 1, {})
    

In [ ]:
# Number of processes
numProcesses = 1

df = [

[0, 1, 2, 0],

[0, 0, 0, 1],

[0, 0, 0, 3],

[0, 0, 0, 0]

]

df = pd.DataFrame(df)

graph = csr_matrix(graph)


# Creating the segments to divide up the dataframe between the threads
segments = np.array_split(np.array(range(0, len(df))), numProcesses)

# Dividing up the threads
dataFrames = []
for i in segments:
    dataFrames.append(df.iloc[i[0]: i[-1] + 1, ])

# Creating the threads
processes = []
manager = mp.Manager()
return_dict = manager.dict()

for process in range(0, numProcesses):
    processes.append(
        mp.Process(target=dijkstrasThread, args=(graph, ['0','1','2','3'], segments[process], process, return_dict))
    )

# Starting the threads
for process in range(0, numProcesses):
    processes[process].start()

# Joining the threads
for process in range(0, numProcesses):
    processes[process].join()

# Putting all the dataframes together into 1
# results = return_dict[0]

# if numProcesses > 1:
#     for process in range(1, numProcesses):
#         results = pd.concat([results, return_dict[process]], axis=0)

In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import dijkstra

# Defining the matrix for scipy
# graph = csr_matrix(timeMatrix.astype(float))

In [ ]:
import time

In [ ]:
# timeMatrix.values

t0 = time.time()

dist_matrix, predecessors = dijkstra(csgraph=graph, directed=False, indices=1, return_predecessors=True)    
t1 = time.time()
print(t1 - t0)



In [ ]:
sum(dist_matrix==-9999)

In [ ]:
print(dist_matrix)
print(predecessors)

86400

In [ ]:
from scipy.sparse import csr_matrix

from scipy.sparse.csgraph import dijkstra



graph = [

[0, 1, 2, 0],

[0, 0, 0, 1],

[0, 0, 0, 3],

[0, 0, 0, 0]

]

graph = csr_matrix(graph)





dist_matrix, predecessors = dijkstra(csgraph=graph, indices=1, directed=False, return_predecessors=True)
# dist_matrix, predecessors = dijkstra(csgraph=graph, directed=False, return_predecessors=True)

print(dist_matrix)

print(predecessors)


In [ ]:
def createRoutes(locations, routes, start):
    completeRoutes = {}
    for i in range(0,len(locations)):
        if i == start:
            completeRoutes[start] = [start]
        else:
            vertex = i
            route = [vertex]
            
            while vertex != start:
                vertex = routes[vertex]
                route.insert(0,vertex)
                completeRoutes[i] = route
                
    return completeRoutes
print(createRoutes([0,1,2,3], predecessors,1))

In [ ]:
def createRoutes(locations, routes):
    traversal = pd.DataFrame(index = locations, columns = locations)
    for i in range(0,len(locations)):
        for j in range(0,len(locations)):
            if i == j:
                traversal.iloc[i,j] = [0]
            else:
                vertex = j
                route = [vertex]
                while vertex != i:
                    vertex = routes[i,vertex]
                    route.insert(0,vertex)
                traversal.iloc[i,j] = route
                
    return traversal
    

    
print(createRoutes([0,1,2,3], predecessors))